## Importação

In [171]:
# Definição do problema
from pulp import *

In [172]:
def relatorio(model, variables, tipo='Minimizar'):
    # Função Objetivo
    print(f'{tipo}:')
    print(f'\tz = {model.objective}')
    print()
    # Restrições
    print('Sujeito a:')
    for restricao in model.constraints.values():
        print(f'\t{restricao}')
    print(f'\t{str(variables)[1:-1]} ∈ [0,1]')
    print()

    # Resolvendo o problema
    status = model.solve()

    # Vendo resultados
    print(f"Status: {'Sucesso' if model.status else 'Falha':}, solução { 'ótima' if LpStatus[model.status] == 'Optimal' else LpStatus[model.status]} encontrada")
    print(f"Valor objetivo (z): {round(model.objective.value(),2)}")
    print()

    print("Valores encontrados:")
    for var in variables:
        print(f"{var.name}: {var.value()}")

## Implementação

### Definição do Modelo

In [174]:
# Definição do modelo
modelo = LpProblem("ExpedicaoPesquisaMarte", sense=LpMinimize)

# Definindo as variaveis de decisão para homens
homens = ("F", "G", "S", "P", "J", "B")
h = {nome: LpVariable(name=f"h{nome}", cat='Binary') for nome in homens} # variável binária

# Definindo as variaveis de decisão para mulheres
mulheres = ("M", "A", "L", "D", "C")
m = {nome: LpVariable(name=f"m{nome}", cat='Binary') for nome in mulheres} # variável binária


### Restrições

In [175]:
# Restrição associada ao número de homens e mulheres na missão:
modelo += lpSum(h[i] for i in homens) >= 2 
modelo += lpSum(m[i] for i in mulheres) >= 1

# Restrição associada à exigência da habilidade de comando:
modelo += lpSum([h['F'], h['G'], h['J'], h['B'], m['D']]) >= 2

# Restrição associada à exigência da habilidade de medicina:
modelo += lpSum([h['G'], m['M'], m['L'], m['C']]) >= 1

# Restrição associada à exigência da habilidade de psicologia:
modelo += lpSum([m['D'], m['C']]) >= 1

# Restrição associada à exigência da habilidade de astronomia:
modelo += lpSum([h['P'], h['B'], h['S']]) >= 1

# Restrição associada à exigência da habilidade de informática:
modelo += lpSum([h['F'], h['J'], m['M'], m['L'], m['D']]) >= 1

# Restrição associada ao uso do simulador:
modelo += lpSum([15*h['F'], 20*h['G'], 18*h['S'], 24*h['P'], 22*h['J'], 10*h['B'], 20*m['M'], 25*m['A'], 20*m['L'], 18*m['D'], 10*m['C']]) <= 1000

# Restrição de compatibilidade pessoal:
modelo += lpSum([h['F'], h['J']]) <= 1
modelo += lpSum([h['B'], m['L']]) <= 1
modelo += lpSum([m['D'], m['A']]) <= 1

# Restrição associada aos aspectos epidemiológicos:
saude_homens = [150, 150, 95, 140, 90, 105]
saude_mulheres = [145, 145, 105, 140, 95]
saude_homens = [saude_homens[idx]*h[nome] for idx, nome in enumerate(homens)]
saude_mulheres = [saude_mulheres[idx]*m[nome] for idx, nome in enumerate(mulheres)]
modelo += sum(saude_mulheres + saude_mulheres) >= 290

# Quantidade de pessoas na expedição
modelo += sum(h.values(), m.values()) == 5

In [176]:
peso_homens = [80, 85, 70, 65, 90, 75]
baguagem_homens = [35, 25, 30, 40, 17, 40]

peso_homens_total = [(peso_homens[idx] + baguagem_homens[idx])*h[nome] for idx, nome in enumerate(homens)]

peso_mulheres = [50, 45, 55, 60, 65]
baguagem_mulheres = [30, 40, 30, 20, 30]

peso_mulheres_total = [(peso_mulheres[idx] + baguagem_mulheres[idx])*m[nome] for idx, nome in enumerate(mulheres)]

In [177]:
# A: É a parcela relativa ao custo pelo peso dos tripulantes e suas bagagens
A = 10**4*sum(peso_homens_total + peso_mulheres_total)

In [179]:
modelo += A

### Resultado

In [180]:
relatorio(modelo, modelo.variables(), 'Minimizar')

Minimizar:
	z = 1150000*hB + 1150000*hF + 1100000*hG + 1070000*hJ + 1050000*hP + 1000000*hS + 850000*mA + 950000*mC + 800000*mD + 850000*mL + 800000*mM

Sujeito a:
	hB + hF + hG + hJ + hP + hS >= 2
	mA + mC + mD + mL + mM >= 1
	hB + hF + hG + hJ + mD >= 2
	hG + mC + mL + mM >= 1
	mC + mD >= 1
	hB + hP + hS >= 1
	hF + hJ + mD + mL + mM >= 1
	10*hB + 15*hF + 20*hG + 22*hJ + 24*hP + 18*hS + 25*mA + 10*mC + 18*mD + 20*mL + 20*mM <= 1000
	hF + hJ <= 1
	hB + mL <= 1
	mA + mD <= 1
	290*mA + 190*mC + 280*mD + 210*mL + 290*mM >= 290
	hB + hF + hG + hJ + hP + hS + mA + mC + mD + mL + mM = 5
	hB, hF, hG, hJ, hP, hS, mA, mC, mD, mL, mM ∈ [0,1]

Status: Sucesso, solução ótima encontrada
Valor objetivo (z): 4520000.0

Valores encontrados:
hB: 0.0
hF: 0.0
hG: 0.0
hJ: 1.0
hP: 0.0
hS: 1.0
mA: 0.0
mC: 0.0
mD: 1.0
mL: 1.0
mM: 1.0
